# The Dataset for Pretraining Word Embedding
:label:`sec_word2vec_data`

In this section, we will introduce how to preprocess a dataset with
negative sampling :numref:`sec_approx_train` and load into minibatches for
word2vec training. The dataset we use is [Penn Tree Bank (PTB)]( https://catalog.ldc.upenn.edu/LDC99T42), which is a small but commonly-used corpus. It takes samples from Wall Street Journal articles and includes training sets, validation sets, and test sets.

First, import the packages and modules required for the experiment.


In [20]:
%use @file[../djl.json]
%use lets-plot
@file:DependsOn("org.apache.commons:commons-math3:3.6.1")
@file:DependsOn("../D2J-1.0-SNAPSHOT.jar")
//import jp.live.ugai.d2j.attention.Chap10Utils
import org.apache.commons.math3.distribution.EnumeratedDistribution
import jp.live.ugai.d2j.util.ImageUtils
import jp.live.ugai.d2j.util.StopWatch
import jp.live.ugai.d2j.util.Training
import jp.live.ugai.d2j.timemachine.Vocab

In [21]:
val manager = NDManager.newBaseManager();

## Reading and Preprocessing the Dataset

This dataset has already been preprocessed. Each line of the dataset acts as a sentence. All the words in a sentence are separated by spaces. In the word embedding task, each word is a token.


In [22]:
fun readPTB() : List<List<String>> {
    val ptbURL = "http://d2l-data.s3-accelerate.amazonaws.com/ptb.zip";
    val input = URL(ptbURL).openStream();
    ZipUtils.unzip(input, Paths.get("./"));

    val lines = mutableListOf<String>();
   val file = File("./ptb/ptb.train.txt");
    val myReader = java.util.Scanner(file);
    while (myReader.hasNextLine()) {
        lines.add(myReader.nextLine());
    }
    val tokens = mutableListOf<List<String>>()
    for (i in 0 until lines.size) {
        tokens.add(lines.get(i).trim().split(" "))
    }
    return tokens;
}

In [23]:
val sentences = readPTB();
println("# sentences: " + sentences.size);

# sentences: 42068


Next we build a vocabulary with words appeared not greater than 10 times mapped into a "&lt;unk&gt;" token. Note that the preprocessed PTB data also contains "&lt;unk&gt;" tokens presenting rare words.


In [24]:
val vocab = Vocab(sentences, 10, listOf<String>())
println(vocab.length());

6719


## Subsampling

In text data, there are generally some words that appear at high frequencies, such "the", "a", and "in" in English. Generally speaking, in a context window, it is better to train the word embedding model when a word (such as "chip") and a lower-frequency word (such as "microprocessor") appear at the same time, rather than when a word appears with a higher-frequency word (such as "the"). Therefore, when training the word embedding model, we can perform subsampling on the words :cite:`Mikolov.Sutskever.Chen.ea.2013`. Specifically, each indexed word $w_i$ in the dataset will drop out at a certain probability. The dropout probability is given as:

$$ P(w_i) = \max\left(1 - \sqrt{\frac{t}{f(w_i)}}, 0\right),$$

Here, $f(w_i)$ is the ratio of the instances of word $w_i$ to the total number of words in the dataset, and the constant $t$ is a hyperparameter (set to $10^{-4}$ in this experiment). As we can see, it is only possible to drop out the word $w_i$ in subsampling when $f(w_i) > t$. The higher the word's frequency, the higher its dropout probability.


In [25]:
fun keep(token: String, counter: Map<Any, Double>, numTokens: Int) : Boolean {
    // Return True if to keep this token during subsampling
    return kotlin.random.Random.nextFloat() < Math.sqrt(1e-4 / counter.get(token)!! * numTokens)
}

fun subSampling(sentences: List<List<String>>, vocab: Vocab) : List<List<String>> {
    val tempSentences = mutableListOf<List<String>>()
    for (i in 0 until sentences.size) {
        val tmp = mutableListOf<String>()
        for (j in 0 until sentences[i].size) {
            tmp.add(vocab.idxToToken.get(vocab.getIdx(sentences[i][j])))
        }
        tempSentences.add(tmp)
    }
    // Count the frequency for each word
    val counter = Vocab.countCorpus2D(sentences);
    var numTokens : Int = 0
    for (value in counter.values) {
        numTokens += value
    }

    // Now do the subsampling
    val output = mutableListOf<List<String>>()
    for (i in 0 until tempSentences.size) {
        val tks = mutableListOf<String>()
        for (j in 0 until tempSentences[i].size) {
            val tk = tempSentences[i][j];
            if (keep(tempSentences[i][j], counter.map { it.key to it.value.toDouble() }.toMap() , numTokens)) {
                tks.add(tk);
            }
        }
        output.add(tks)
    }

    return output
}

val subsampled = subSampling(sentences, vocab);

Compare the sequence lengths before and after sampling, we can see subsampling significantly reduced the sequence length.


In [26]:
val y1 = sentences.map { it.size }
val y2 = subsampled.map { it.size }

val label = List<String>(sentences.size){"body"}
    val label2 = List<String>(subsampled.size){"subsampled"}
val data = mapOf("x" to y1 + y2, "cond" to label + label2)
var plot = letsPlot(data){x = "x"; fill="cond"} 
plot += geomHistogram(binWidth=5)
plot + ggsize(500, 250)
//println(y1)
// HistogramTrace trace1 =
//        HistogramTrace.builder(y1).opacity(.75).name("origin").nBinsX(20).build();
// HistogramTrace trace2 =
//         HistogramTrace.builder(y2).opacity(.75).name("subsampled").nBinsX(20).build();

// Layout layout =
//        Layout.builder()
//                .barMode(Layout.BarMode.GROUP)
//                .showLegend(true)
//                .xAxis(Axis.builder().title("# tokens per sentence").build())
//                .yAxis(Axis.builder().title("count").build())
//                .build();
//new Figure(layout, trace1, trace2);

For individual tokens, the sampling rate of the high-frequency word "the" is less than 1/20.


In [27]:
fun compareCounts(token: String, sentences: List<List<String>>, subsampled: List<List<String>>): String {
    var beforeCount = 0;
    for (i in 0 until sentences.size) {
            beforeCount += sentences[i].count{ it.equals(token) } 
    }

    var afterCount = 0;
    for (i in 0 until subsampled.size) {
        afterCount += subsampled[i].count { it.equals(token) }
    }

    return "# of \"the\": before=" + beforeCount + ", after=" + afterCount;
}

println(compareCounts("the", sentences, subsampled));

# of "the": before=50770, after=2128


But the low-frequency word "join" is completely preserved.


In [28]:
println(compareCounts("join", sentences, subsampled));

# of "the": before=45, after=45


Last, we map each token into an index to construct the corpus.


In [29]:
val corpus = mutableListOf<List<Int>>()
for (i in 0 until subsampled.size) {
    corpus.add(vocab.getIdxs(subsampled[i]))
}
for (i in 0 until 3) {
    println(corpus[i])
}

[0]
[2115, 145]
[5277, 3054, 1580]


## Loading the Dataset

Next we read the corpus with token indicies into data batches for training.

### Extracting Central Target Words and Context Words

We use words with a distance from the central target word not exceeding the context window size as the context words of the given center target word. The following definition function extracts all the central target words and their context words. It uniformly and randomly samples an integer to be used as the context window size between integer 1 and the `maxWindowSize` (maximum context window).


In [30]:
fun  getCentersAndContext(
        corpus: List<List<Int>>, maxWindowSize: Int): kotlin.Pair<List<Int>, List<List<Int>>> {
    var centers = mutableListOf<Int>()
    val contexts = mutableListOf<List<Int>>()

    for (line in corpus) {
        // Each sentence needs at least 2 words to form a "central target word
        // - context word" pair
        if (line.size < 2) {
            continue;
        }
        centers.addAll(line)
        for (i in 0 until line.size) { // Context window centered at i
            val windowSize = kotlin.random.Random.nextInt(maxWindowSize - 1) + 1;
            val indices =
                    (Math.max(0, i - windowSize) until Math.min(line.size, i + 1 + windowSize)).toMutableList()
            // Exclude the central target word from the context words
            indices.remove(indices.indexOf(i))
            val context = indices.map { line[it] }
            contexts.add(context);
        }
    }
    return kotlin.Pair(centers, contexts);
}

Next, we create an artificial dataset containing two sentences of 7 and 3 words, respectively. Assume the maximum context window is 2 and print all the central target words and their context words.


In [31]:
val tinyDataset =
        listOf<List<Int>>(
            (0 until 7).toList(),
            (7 until 10).toList())

println("dataset " +tinyDataset)
var centerContextPair =
        getCentersAndContext(tinyDataset, 2);
println(centerContextPair)
for (i in 0 until centerContextPair.second.size) {
    println(
            "Center "
                    + centerContextPair.first.get(i)
                    + " has contexts"
                    + centerContextPair.second.get(i))
}

dataset [[0, 1, 2, 3, 4, 5, 6], [7, 8, 9]]
([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], [[1], [0, 2], [2, 3], [2, 3, 4], [3, 4, 5], [4, 5, 6], [5, 6], [8], [7, 9], [9]])
Center 0 has contexts[1]
Center 1 has contexts[0, 2]
Center 2 has contexts[2, 3]
Center 3 has contexts[2, 3, 4]
Center 4 has contexts[3, 4, 5]
Center 5 has contexts[4, 5, 6]
Center 6 has contexts[5, 6]
Center 7 has contexts[8]
Center 8 has contexts[7, 9]
Center 9 has contexts[9]


We set the maximum context window size to 5. The following extracts all the central target words and their context words in the dataset.


In [32]:
centerContextPair = getCentersAndContext(corpus, 5);
val allCenters = centerContextPair.first
val allContexts = centerContextPair.second
println("# center-context pairs:" + allCenters.size)

# center-context pairs:353384


### Negative Sampling

We use negative sampling for approximate training. For a central and context word pair, we randomly sample $K$ noise words ($K=5$ in the experiment). According to the suggestion in the Word2vec paper, the noise word sampling probability $P(w)$ is the ratio of the word frequency of $w$ to the total word frequency raised to the power of 0.75 :cite:`Mikolov.Sutskever.Chen.ea.2013`.

We first define a class to draw a candidate according to the sampling weights. It caches a 10,000 size random number bank.


In [44]:
class RandomGenerator(samplingWeights: List<Double> ) {
    /* Draw a random int in [0, n] according to n sampling weights. */

    private val population: kotlin.collections.List<Int>
    private val samplingWeights: kotlin.collections.List<Double> 
    private var candidates: MutableList<Int> 
    private val pmf: MutableList<org.apache.commons.math3.util.Pair<Int, Double>>
    private var i: Int

    init {
        this.population =
                (0 until samplingWeights.size).toList()
        this.samplingWeights = samplingWeights
        this.candidates = mutableListOf()
        this.i = 0;

        this.pmf = mutableListOf()
        for (i in 0 until samplingWeights.size) {
            this.pmf.add(org.apache.commons.math3.util.Pair(this.population.get(i), this.samplingWeights.get(i)));
        }
    }

    fun draw(): Int {
        if (this.i == this.candidates.size) {
            this.candidates =
                    EnumeratedDistribution(this.pmf).sample(10000, arrayOf<Int>()).toMutableList()
            this.i = 0;
        }
        this.i += 1;
        return this.candidates.get(this.i - 1);
    }
}

val generator =
        RandomGenerator(listOf(2.0, 3.0, 4.0))
val generatorOutput = List(10) { generator.draw() }
println(generatorOutput)

[2, 2, 2, 0, 2, 0, 1, 2, 1, 1]


In [49]:
fun getNegatives(allContexts: List<List<Int>>, corpus: List<List<Int>>, K: Int): List<List<Int>> {
    val counter = Vocab.countCorpus2D(corpus)
    val samplingWeights = mutableListOf<Double>()
    for (entry in counter) {
        samplingWeights.add(Math.pow(entry.value.toDouble(), 0.75));
    }
    val allNegatives = mutableListOf<List<Int>>()
    val generator = RandomGenerator(samplingWeights)
    for (contexts in allContexts) {
        val negatives = mutableListOf<Int>()
        while (negatives.size < contexts.size * K) {
            val neg = generator.draw();
            // Noise words cannot be context words
            if (!contexts.contains(neg)) {
                negatives.add(neg);
            }
        }
        allNegatives.add(negatives);
    }
    return allNegatives;
}

val allNegatives = getNegatives(allContexts, corpus, 5);

### Reading into Batches

We extract all central target words `allCenters`, and the context words `allContexts` and noise words `allNegatives` of each central target word from the dataset. We will read them in random minibatches.

In a minibatch of data, the $i^\mathrm{th}$ example includes a central word and its corresponding $n_i$ context words and $m_i$ noise words. Since the context window size of each example may be different, the sum of context words and noise words, $n_i+m_i$, will be different. When constructing a minibatch, we concatenate the context words and noise words of each example, and add 0s for padding until the length of the concatenations are the same, that is, the length of all concatenations is $\max_i n_i+m_i$(`maxLen`). In order to avoid the effect of padding on the loss function calculation, we construct the mask variable `masks`, each element of which corresponds to an element in the concatenation of context and noise words, `contextsNegatives`. When an element in the variable `contextsNegatives` is a padding, the element in the mask variable `masks` at the same position will be 0. Otherwise, it takes the value 1. In order to distinguish between positive and negative examples, we also need to distinguish the context words from the noise words in the `contextsNegatives` variable. Based on the construction of the mask variable, we only need to create a label variable `labels` with the same shape as the `contextsNegatives` variable and set the elements corresponding to context words (positive examples) to 1, and the rest to 0.

Next, we will implement the minibatch reading function `batchifyData`. Its minibatch input `data` is a list of `NDArrays`, each element of which contains central target words `center`, context words `context`, and noise words `negative`. The minibatch data returned by this function conforms to the format we need, for example, it includes the mask variable.


In [56]:
fun batchifyData(data: List<NDList>) : NDList{
    val centers = NDList();
    val contextsNegatives = NDList();
    val masks = NDList();
    val labels = NDList();

    var maxLen = 0L
    for (ndList in data) { // center, context, negative = ndList
        maxLen =
                Math.max(
                        maxLen,
                        ndList.get(1).countNonzero().getLong()
                                + ndList.get(2).countNonzero().getLong());
    }
    for (ndList in data) { // center, context, negative = ndList
        val center = ndList.get(0);
        val context = ndList.get(1);
        val negative = ndList.get(2);

        var count = 0L;
        for (i in 0 until context.size()) {
            // If a 0 is found, we want to stop adding these
            // values to NDArray
            if (context.get(i).getInt() == 0) {
                break;
            }
            contextsNegatives.add(context.get(i).reshape(1));
            masks.add(manager.create(1).reshape(1));
            labels.add(manager.create(1).reshape(1));
            count += 1;
        }
        for (i in 0 until negative.size()) {
            // If a 0 is found, we want to stop adding these
            // values to NDArray
            if (negative.get(i).getInt() == 0) {
                break;
            }
            contextsNegatives.add(negative.get(i).reshape(1));
            masks.add(manager.create(1).reshape(1));
            labels.add(manager.create(0).reshape(1));
            count += 1;
        }
        // Fill with zeroes remaining array
        while (count != maxLen) {
            contextsNegatives.add(manager.create(0).reshape(1));
            masks.add(manager.create(0).reshape(1));
            labels.add(manager.create(0).reshape(1));
            count += 1;
        }

        // Add this NDArrays to output NDArrays
        centers.add(center.reshape(1));
    }
    return NDList(
            NDArrays.concat(centers).reshape(data.size.toLong(), -1),
            NDArrays.concat(contextsNegatives).reshape(data.size.toLong(), -1),
            NDArrays.concat(masks).reshape(data.size.toLong(), -1),
            NDArrays.concat(labels).reshape(data.size.toLong(), -1));
}

Construct two simple examples:


In [59]:
val x1 =
        NDList(
                manager.create(intArrayOf(1)),
                manager.create(intArrayOf(2, 2)),
                manager.create(intArrayOf(3, 3, 3, 3)))
val x2 =
        NDList(
                manager.create(intArrayOf(1)),
                manager.create(intArrayOf(2, 2, 2)),
                manager.create(intArrayOf(3, 3)))

val batchedData = batchifyData(listOf<NDList>(x1, x2));
val names = listOf("centers", "contexts_negatives", "masks", "labels")
for (i in 0 until batchedData.size) {
    println(names[i] + " shape: " + batchedData.get(i));
}

centers shape: ND: (2, 1) cpu() int32
[[ 1],
 [ 1],
]

contexts_negatives shape: ND: (2, 6) cpu() int32
[[ 2,  2,  3,  3,  3,  3],
 [ 2,  2,  2,  3,  3,  0],
]

masks shape: ND: (2, 6) cpu() int32
[[ 1,  1,  1,  1,  1,  1],
 [ 1,  1,  1,  1,  1,  0],
]

labels shape: ND: (2, 6) cpu() int32
[[ 1,  1,  0,  0,  0,  0],
 [ 1,  1,  1,  0,  0,  0],
]



We use the `batchifyData` function just defined to specify the minibatch reading method for the `ArrayDataset` instance iterator.

## Putting All Things Together

Last, we define the `loadDataPTB` function that read the PTB dataset and return the dataset. In addition, we will create a function called `convertNDArray` that will convert the `centers`, `contexts`, and `negatives` lists, into `NDArrays` by putting 0s where there is no data in order for the rows to have the same lenghts.


In [74]:
fun convertNDArray(data: List<List<Any>>, manager: NDManager): NDList {
    val centers: MutableList<Int> = (data[0] as List<Int>).toMutableList()
    val contexts: List<MutableList<Int>> = (data[1] as List<List<Int>>).map { it.toMutableList() }
    val negatives: List<MutableList<Int>> = (data[2] as List<List<Int>>).map { it.toMutableList() }

    // Create centers NDArray
    val centersNDArray = manager.create(centers.toIntArray());

    // Create contexts NDArray
    var maxLen = 0;
    for (context in contexts) {
        maxLen = Math.max(maxLen, context.size)
    }
    // Fill arrays with 0s to all have same lengths and be able to create NDArray
    for (context in contexts) {
        while (context.size != maxLen) {
            context.add(0);
        }
    }
    val contextsNDArray = manager.create(contexts.map { it.toIntArray() }.toTypedArray());

    // Create negatives NDArray
    maxLen = 0;
    for (negative in negatives) {
        maxLen = Math.max(maxLen, negative.size)
    }
    // Fill arrays with 0s to all have same lengths and be able to create NDArray
    for (negative in negatives) {
        while (negative.size != maxLen) {
            negative.add(0);
        }
    }
    val negativesNDArray =
            manager.create(
                    negatives.map { it.toIntArray() }.toTypedArray());

    return NDList(centersNDArray, contextsNDArray, negativesNDArray);
}

    fun loadDataPTB(
        batchSize: Int,
        maxWindowSize: Int,
        numNoiseWords: Int,
        manager: NDManager
    ): Pair<ArrayDataset, Vocab> {
        val sentences = readPTB()
        val vocab = Vocab(sentences, 10, listOf<String>())
        val subSampled = subSampling(sentences, vocab)
        val corpus = mutableListOf<List<Int>>()
        for (i in 0 until subSampled.size) {
            corpus.add(vocab.getIdxs(subSampled[i]))
        }
        val pair = getCentersAndContext(corpus, maxWindowSize)
        val negatives = getNegatives(pair.second, corpus, numNoiseWords)

        val ndArrays =
            convertNDArray(listOf(pair.first, pair.second, negatives), manager)
        val dataset =
            ArrayDataset.Builder()
                .setData(ndArrays.get(0), ndArrays.get(1), ndArrays.get(2))
                .optDataBatchifier(
                    object : Batchifier {
                        override fun batchify(ndLists: Array<NDList>): NDList {
                            return batchifyData(ndLists.toList())
                        }

                        override fun unbatchify(ndList: NDList): Array<NDList> {
                            return arrayOf<NDList>()
                        }
                    })
                .setSampling(batchSize, true)
                .build()

        return Pair(dataset, vocab)
    }


Let us print the first minibatch of the dataset.


In [75]:
val datasetVocab = loadDataPTB(512, 5, 5, manager);
val dataset = datasetVocab.getKey();
val vocab = datasetVocab.getValue();

val batch = dataset.getData(manager).iterator().next();
for (i in 0 until batch.getData().size) {
    println(names[i] + " shape: " + batch.getData().get(i).getShape());
}

centers shape: (512, 1)
contexts_negatives shape: (512, 54)
masks shape: (512, 54)
labels shape: (512, 54)


## Summary

* Subsampling attempts to minimize the impact of high-frequency words on the training of a word embedding model.
* We can pad examples of different lengths to create minibatches with examples of all the same length and use mask variables to distinguish between padding and non-padding elements, so that only non-padding elements participate in the calculation of the loss function.

## Exercises

1. We use the `batchifyData` function to specify the minibatch reading method for the `ArrayDataset` instance iterator and print the shape of each variable in the first batch read. How should these shapes be calculated?
